In [1]:
import pandas as pd
import time

In [2]:
id = "1t_bpJX7PC0dqZ5pEmaxM8wmDJteYmOXB" # google file ID
df = pd.read_csv("https://docs.google.com/uc?id=" + id, sep = ';')
df.head()

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en


In [3]:
import re
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
from nltk.corpus import wordnet

stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

def clean_text(text):
    if isinstance(text, str):
        # delete websites
        text_noweb = re.sub('http\\S+', '', text)
        # delete numbers
        text_nonum = re.sub(r'\d+', '', text_noweb)
        # make a list of tokens
        try:
            text_token = word_tokenize(text_nonum, language='english')
        except IndexError:
            try:
                text_token = word_tokenize(text_nonum[1:], language='english')
            except:
                text_token = []

        # add words that don't contain special symbols
        clean_words = []
        for word in text_token:
            if word.isalnum():
                clean_words.append(word)

        # delete stopwords and make lowercase
        words_nostop = [word.lower() for word in clean_words if word.lower() not in stop_words]

        # get the stem of the word
        words_stem = [stemmer.stem(word) for word in words_nostop]
        
        return ' '.join(words_stem)
    else:
        return ''

In [5]:
df['title_clean'] = df.title.apply(lambda x: clean_text(x))
df.head()

,topic,link,domain,published_date,title,lang,title_clean
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,closer look solar fuel potenti
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,irresist scent make locust swarm studi find
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,artifici intellig warn ai know us better know
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,glacier could sculpt mar valley studi
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,perseid meteor shower time see huge bright fir...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=8)

X = df.title_clean
y, uniques = pd.factorize(df.topic)

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size = 0.01, random_state = 0)

X_train = vectorizer.fit_transform(X_train)
X_val = vectorizer.transform(X_val)
X_test = vectorizer.transform(X_test)

X_train.shape

(87019, 8372)

# Logistic Regression

## Multinomial classification

In [33]:
log0 = LogisticRegressionCV(Cs=100, cv=3, random_state=42, max_iter=1000, 
                            multi_class='multinomial', solver='lbfgs', 
                            n_jobs=-1, penalty='l2')
log0.fit(X_train, y_train)

print(log0.score(X_train, y_train))
print(log0.score(X_val, y_val))

0.858065479952654
0.7901286158703626


In [35]:
print(log0.scores_)

{0: array([[0.24587169, 0.28244906, 0.32071569, 0.36029234, 0.40379908,
        0.44747819, 0.49053677, 0.52876892, 0.56234702, 0.59013342,
        0.61126625, 0.62705554, 0.6372255 , 0.64353432, 0.64618885,
        0.64846416, 0.64918813, 0.64984314, 0.65087737, 0.65294584,
        0.65401455, 0.65480746, 0.65680698, 0.65880649, 0.66146103,
        0.66377081, 0.66663219, 0.66983831, 0.67311339, 0.67766401,
        0.68448995, 0.6904885 , 0.69707312, 0.70365774, 0.70982866,
        0.71644775, 0.72313579, 0.72889303, 0.73471921, 0.7398559 ,
        0.74488916, 0.74930189, 0.7531975 , 0.75678285, 0.76005792,
        0.76329851, 0.76684938, 0.76988313, 0.77188265, 0.77453718,
        0.77601958, 0.77777778, 0.77822595, 0.7793636 , 0.78019099,
        0.77967387, 0.77843279, 0.77805357, 0.77812252, 0.77719171,
        0.77522667, 0.7732961 , 0.77191712, 0.77029682, 0.76771124,
        0.76512566, 0.76302272, 0.7610232 , 0.75888579, 0.75540387,
        0.75247354, 0.74995691, 0.74795739, 

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [ ]:
print(log0.C_)

## One Vs Rest approach

In [20]:
%%time
from sklearn.linear_model import LogisticRegressionCV
Cs = [100.0, 50.0, 10.0, 1.0, 0.1, 0.01, 0.001]
log1 = LogisticRegressionCV(Cs=100, cv=3, random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', n_jobs=-1, 
                            penalty='l1')
log1.fit(X_train, y_train)

print(log1.score(X_train, y_train))
print(log1.score(X_val, y_val))

0.8727864029694664
0.7818637693272044
CPU times: user 15.2 s, sys: 122 ms, total: 15.3 s
Wall time: 1min 29s


In [25]:
log1 = LogisticRegressionCV(Cs=100, cv=3, random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', 
                            n_jobs=-1, penalty='l1')
log1.fit(X_train, y_train)

print(log1.score(X_train, y_train))
print(log1.score(X_val, y_val))

0.860984382721015
0.7908715234248038


In [26]:
log1.Cs_

array([1.00000000e-04, 1.20450354e-04, 1.45082878e-04, 1.74752840e-04,
       2.10490414e-04, 2.53536449e-04, 3.05385551e-04, 3.67837977e-04,
       4.43062146e-04, 5.33669923e-04, 6.42807312e-04, 7.74263683e-04,
       9.32603347e-04, 1.12332403e-03, 1.35304777e-03, 1.62975083e-03,
       1.96304065e-03, 2.36448941e-03, 2.84803587e-03, 3.43046929e-03,
       4.13201240e-03, 4.97702356e-03, 5.99484250e-03, 7.22080902e-03,
       8.69749003e-03, 1.04761575e-02, 1.26185688e-02, 1.51991108e-02,
       1.83073828e-02, 2.20513074e-02, 2.65608778e-02, 3.19926714e-02,
       3.85352859e-02, 4.64158883e-02, 5.59081018e-02, 6.73415066e-02,
       8.11130831e-02, 9.77009957e-02, 1.17681195e-01, 1.41747416e-01,
       1.70735265e-01, 2.05651231e-01, 2.47707636e-01, 2.98364724e-01,
       3.59381366e-01, 4.32876128e-01, 5.21400829e-01, 6.28029144e-01,
       7.56463328e-01, 9.11162756e-01, 1.09749877e+00, 1.32194115e+00,
       1.59228279e+00, 1.91791026e+00, 2.31012970e+00, 2.78255940e+00,
      

In [27]:
log1.scores_

{0: array([[0.96573241, 0.96573241, 0.96573241, 0.96573241, 0.96573241,
         0.96573241, 0.96573241, 0.96573241, 0.96573241, 0.96573241,
         0.96573241, 0.96573241, 0.96573241, 0.96573241, 0.96573241,
         0.96573241, 0.96573241, 0.96573241, 0.96573241, 0.96573241,
         0.96573241, 0.96573241, 0.96573241, 0.96573241, 0.96573241,
         0.96573241, 0.96573241, 0.96583583, 0.96652532, 0.96742166,
         0.96818009, 0.96893853, 0.97011066, 0.97124832, 0.97252387,
         0.97386838, 0.97517841, 0.97631606, 0.97662633, 0.97748819,
         0.97838453, 0.97900507, 0.97979798, 0.98059089, 0.98090116,
         0.98221119, 0.98293515, 0.98348674, 0.98379701, 0.98434861,
         0.98507257, 0.98531389, 0.98555521, 0.98614128, 0.9863826 ,
         0.98679629, 0.98696866, 0.98738236, 0.9875892 , 0.98724446,
         0.98714103, 0.98689971, 0.98683076, 0.9863826 , 0.98621023,
         0.98572758, 0.98483125, 0.98458993, 0.98400386, 0.98334885,
         0.98321095, 0.98293515

In [28]:
log0 = LogisticRegressionCV(Cs=Cs, cv=3, random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', 
                            n_jobs=-1, penalty='l2')
log0.fit(X_train, y_train)

print(log0.score(X_train, y_train))
print(log0.score(X_val, y_val))

0.8668106965145543
0.7913822723684821


In [29]:
log0.get_params()

{'Cs': [100.0, 50.0, 10.0, 1.0, 0.1, 0.01, 0.001],
 'class_weight': None,
 'cv': 3,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 10000,
 'multi_class': 'ovr',
 'n_jobs': -1,
 'penalty': 'l2',
 'random_state': 42,
 'refit': True,
 'scoring': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0}

In [30]:
log0 = LogisticRegressionCV(Cs=100, cv=3, random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', 
                            n_jobs=-1, penalty='l2')
log0.fit(X_train, y_train)

print(log0.score(X_train, y_train))
print(log0.score(X_val, y_val))

0.8631563221825118
0.792589497144449


### SVM

In [36]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

grid_params = {
    'C': [1000, 100, 10, 1, 0.1, 0.01, 0.001],
}

svm1 = LinearSVC(random_state=42, penalty='l1', dual=False, max_iter=10000)
svm_cv1 = GridSearchCV(svm1, param_grid=grid_params, n_jobs = -1, cv=4)
svm_cv1.fit(X_train, y_train)

print(svm_cv1.score(X_train, y_train))
print(svm_cv1.score(X_val, y_val))
print(svm_cv1.best_params_)
print(svm_cv1.best_score_)

0.8558820487479746
0.7929609509216697
{'C': 1}
0.7821280788183586


In [37]:
svm0 = LinearSVC(random_state=42, penalty='l2', dual=False, max_iter=10000)
svm_cv0= GridSearchCV(svm0, param_grid=grid_params, n_jobs = -1, cv=4)
svm_cv0.fit(X_train, y_train)

print(svm_cv0.score(X_train, y_train))
print(svm_cv0.score(X_val, y_val))
print(svm_cv0.best_params_)
print(svm_cv0.best_score_)

0.8644778726485021
0.793471699865348
{'C': 1}
0.7841391435167102


/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

# Word Embeddings

In [2]:
import pandas as pd
id = "1kgfBvHHJ-uWnth6GKP89m9dJhof3fMUu" # google file ID
df = pd.read_csv("https://docs.google.com/uc?id=" + id + "&confirm=t", sep=',')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,topic
0,0.457132,-0.412739,-0.932816,0.728594,-0.209356,-0.106480,0.963394,0.047319,-0.811320,-0.951418,...,0.232601,0.884456,0.278755,0.345579,-0.901508,0.679105,-0.398569,0.195617,0.512678,5
1,-0.542360,0.503637,0.261783,-0.093952,0.605817,0.052741,-0.842242,-0.257135,-0.473485,-0.009523,...,-0.047023,0.947700,-0.854258,-0.886906,-0.135484,-0.497849,-0.578527,0.170515,-0.614010,0
2,-0.414353,0.600139,0.733021,0.290133,-0.559828,0.247470,0.082544,-0.441254,-0.454951,-0.268327,...,0.566598,-0.258558,0.376647,-0.331484,0.336149,0.137495,0.617308,-0.112076,-0.117658,2
3,-0.005987,-0.137958,0.854867,0.457844,-0.386081,-0.126676,0.182546,0.098479,0.755384,0.564700,...,0.223361,-0.761614,0.346789,0.130925,0.698150,0.628125,0.782671,-0.467192,0.335435,1
4,0.462713,-0.754823,-0.999642,0.901334,0.466083,-0.112792,0.991975,0.453595,-0.995754,-0.973586,...,-0.952645,0.999592,-0.292417,0.182599,-0.994657,0.607546,-0.984408,0.115027,0.373930,5


In [7]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, 0:768]
y, uniques = pd.factorize(df.topic)

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size = 0.01, random_state = 0)

X_train.shape

(43509, 768)

In [ ]:
%%time
log = LogisticRegression(random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', 
                            n_jobs=-1, penalty='l1')
log.fit(X_train, y_train)

print(log.score(X_train, y_train))
print(log.score(X_val, y_val))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1526: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs))


In [40]:
%%time
log1 = LogisticRegressionCV(Cs=100, cv=3, random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', 
                            n_jobs=-1, penalty='l1')
log1.fit(X_train, y_train)

print(log1.score(X_train, y_train))
print(log1.score(X_val, y_val))

KeyboardInterrupt: 

In [41]:
%%time
log0 = LogisticRegressionCV(Cs=100, cv=3, random_state=42, max_iter=10000, 
                            multi_class='ovr', solver='liblinear', 
                            n_jobs=-1, penalty='l2')
log0.fit(X_train, y_train)

print(log0.score(X_train, y_train))
print(log0.score(X_val, y_val))

KeyboardInterrupt: 

In [4]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

In [9]:
%%time
svm = LinearSVC(random_state=42, dual=False, max_iter=10000)
svm.fit(X_train, y_train)

print(svm.score(X_train, y_train))
print(svm.score(X_val, y_val))

KeyboardInterrupt: 

In [ ]:
%%time
grid_params = {
    'C': [100, 50, 10, 1, 0.1, 0.01, 0.001],
    'penalty': ['l1', 'l2']
}

svm = LinearSVC(random_state=42, dual=False, max_iter=10000)
svm_cv = GridSearchCV(svm, param_grid=grid_params, n_jobs = -1, cv=4)
svm_cv.fit(X_train, y_train)

print(svm_cv.score(X_train, y_train))
print(svm_cv.score(X_val, y_val))
print(svm_cv.best_params_)